<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-03 13:19:38
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.37 C
-------------------
Today PnL: 42.58 K (0.31%)
Current PnL: -13.16 L (-9.17%)
CY Booked + Current PnL: -6.42 L (-4.47%)
-------------------
Total profit:  4.76 L
Total loss:  -17.92 L
-------------------
Total Booked + Current PnL: 20.56 L (17.64%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.93%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.59 L (52.35%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.88%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.00,54.03,M-SC,90.62,133299.0,-9180.0,9797.0,-1.06,-6.44,7.35,0.44,243.0,-0.94,0.98,57.33,OX40N,NTT,BANKS
39,HINDALCO,651.95,761.55,-19.08,H-LC,86.46,111744.0,7432.0,10102.0,0.01,7.12,9.04,16.81,27.0,0.74,0.82,24.31,X5K,ATH,METALS
3,ADANIPORTS,1103.69,1583.00,1.31,M-LC,36.46,101842.0,23480.0,10551.0,-0.66,29.96,10.36,43.43,77.0,2.23,0.75,36.88,XY24,NTT,MISC
5,APOLLOHOSP,6904.43,8285.00,-16.63,H-LC,88.54,113595.0,10029.0,10678.0,1.77,9.68,9.40,20.00,30.0,0.94,0.83,25.12,X40N,BTT,HEALTHCARE
88,UNIONBANK,123.87,163.00,-1.39,M-LC,26.04,171664.0,30824.0,13664.0,-1.57,21.89,7.96,31.59,89.0,2.26,1.26,49.12,XY24,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,142448.0,606.0,55968.0,0.05,0.43,39.29,39.88,259.0,0.01,1.04,44.36,XY24,NTT,POWER
12,BANKINDIA,116.91,190.00,-31.29,M-MC,17.71,181561.0,1753.0,110661.0,-1.01,0.98,60.95,62.52,171.0,0.02,1.33,35.39,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,46.88,151515.0,602.0,59848.0,2.01,0.40,39.50,40.06,102.0,0.01,1.11,23.93,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,HONAUT,42646.15,58357.33,-15.36,X-MC,65.62,122115.0,-5823.0,52961.0,4.87,-4.55,43.37,36.84,56.0,-0.11,0.89,25.02,X40N,ATH,ELECTRICAL
15,BLUESTARCO,1646.70,2337.55,-2.21,H-SC,97.92,183350.0,18680.0,50403.0,4.41,11.34,27.49,41.95,121.0,0.37,1.34,19.66,X40N,ATH,AC
93,VOLTAS,1278.28,1918.49,-24.38,H-MC,89.58,205800.0,14058.0,81970.0,2.94,7.33,39.83,50.08,100.0,0.17,1.51,14.63,XY25,ATH,AC
59,MEDANTA,1064.65,1486.00,7.69,X-SC,16.67,229670.0,24193.0,57119.0,2.05,11.77,24.87,39.58,65.0,0.42,1.68,23.07,XY24,NTT,HEALTHCARE
36,HAVELLS,1588.02,2069.16,-4.46,X-MC,61.46,246730.0,-1001.0,76067.0,2.01,-0.40,30.83,30.30,53.0,-0.01,1.81,12.83,X40,ATH,ELECTRICAL


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,2.08,16401.0,-33192.0,2116.0,-2.33,-66.93,12.90,-62.66,NaN,-15.69,0.12,9.99,XY24,ATH,MISC
44,ICICIPRULI,600.83,790.00,-9.93,H-MC,35.42,145499.0,9711.0,33043.0,-2.28,7.15,22.71,31.48,119.0,0.29,1.06,20.14,X40,ATH,INSURANCE
11,BANDHANBNK,214.25,400.00,-28.38,H-SC,78.12,213417.0,-35113.0,250573.0,-2.08,-14.13,117.41,86.70,129.0,-0.14,1.56,41.19,XY24,NTT,BANKS
88,UNIONBANK,123.87,163.00,-1.39,M-LC,26.04,171664.0,30824.0,13664.0,-1.57,21.89,7.96,31.59,89.0,2.26,1.26,49.12,XY24,NTT,BANKS
92,VIPIND,488.80,489.00,-212.13,H-SC,95.83,80896.0,-14420.0,14456.0,-1.23,-15.13,17.87,0.04,154.0,-1.00,0.59,59.56,OX40N,NTT,MISC


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,54.03,M-SC,90.62,133299.0,-9180.0,9797.0,-1.06,-6.44,7.35,0.44,243.0,-0.94,0.98,57.33,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-30.89,M-SC,8.33,147231.0,-23910.0,23851.0,1.55,-13.97,16.20,-0.03,189.0,-1.00,1.08,5.82,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,92.71,148912.0,-25647.0,25687.0,0.60,-14.69,17.25,0.02,127.0,-1.00,1.09,29.13,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-16.64,X-MC,22.92,87514.0,-15138.0,15131.0,1.07,-14.75,17.29,-0.00,58.0,-1.00,0.64,33.03,OX40N,NTT,IT
92,VIPIND,488.80,489.0,-212.13,H-SC,95.83,80896.0,-14420.0,14456.0,-1.23,-15.13,17.87,0.04,154.0,-1.00,0.59,59.56,OX40N,NTT,MISC


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,6.25,47727.0,6472.0,4334.0,-0.10,15.69,9.08,26.19,1.0,1.49,0.35,12.36,XY24,ATH,IT
83,TCS,3794.03,4998.00,-18.38,X-LC,42.71,266206.0,-29728.0,123626.0,-0.30,-10.05,46.44,31.73,1.0,-0.24,1.95,6.55,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.19,X-LC,63.54,290875.0,29274.0,116350.0,0.90,11.19,40.00,55.67,2.0,0.25,2.13,17.98,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-17.03,X-LC,28.12,259907.0,-10440.0,45068.0,0.59,-3.86,17.34,12.81,6.0,-0.23,1.90,8.70,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-11.71,X-LC,34.38,280098.0,14672.0,42239.0,0.25,5.53,15.08,21.44,9.0,0.35,2.05,12.07,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
94,WHIRLPOOL,1167.49,2270.0,-37.42,M-SC,93.75,206153.0,34532.0,127547.0,1.11,20.12,61.87,94.43,198.0,0.27,1.51,52.15,XR,NTT,DURABLES
88,UNIONBANK,123.87,163.0,-1.39,M-LC,26.04,171664.0,30824.0,13664.0,-1.57,21.89,7.96,31.59,89.0,2.26,1.26,49.12,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,1.31,M-LC,36.46,101842.0,23480.0,10551.0,-0.66,29.96,10.36,43.43,77.0,2.23,0.75,36.88,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,69.79,72992.0,38543.0,1401.0,-1.06,111.89,1.92,115.97,NaN,27.51,0.53,11.62,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,96.05,M-MC,59.38,235673.0,10711.0,156935.0,-0.45,4.76,66.59,74.53,185.0,0.07,1.72,36.83,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,26.30,H-SC,68.75,129188.0,3891.0,55318.0,-0.42,3.11,42.82,47.25,139.0,0.07,0.95,61.62,MH,ATH,POWER
20,CAMS,3643.00,5250.99,6.09,H-SC,44.79,119221.0,17217.0,27802.0,0.64,16.88,23.32,44.14,124.0,0.62,0.87,35.86,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,1.31,M-LC,36.46,101842.0,23480.0,10551.0,-0.66,29.96,10.36,43.43,77.0,2.23,0.75,36.88,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,26.04,171664.0,30824.0,13664.0,-1.57,21.89,7.96,31.59,89.0,2.26,1.26,49.12,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,69.79,72992.0,38543.0,1401.0,-1.06,111.89,1.92,115.97,NaN,27.51,0.53,11.62,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-37.42,M-SC,93.75,206153.0,34532.0,127547.0,1.11,20.12,61.87,94.43,198.0,0.27,1.51,52.15,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.41,M-LC,83.33,166811.0,15866.0,93581.0,0.77,10.51,56.10,72.51,69.0,0.17,1.22,14.90,XR,NTT,IT
45,INDIAMART,2327.09,4871.06,-47.89,H-SC,84.38,135839.0,12503.0,122323.0,-0.25,10.14,90.05,109.32,122.0,0.10,0.99,34.20,AR,ATH,MISC
54,KPIGREEN,497.21,732.16,26.30,H-SC,68.75,129188.0,3891.0,55318.0,-0.42,3.11,42.82,47.25,139.0,0.07,0.95,61.62,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68122.0,-45427.0,85479.0,-0.11,-40.01,125.48,35.27,261.0,-0.53,0.50,66.73,XR,NTT,HOTELS
58,MASFIN,326.60,402.39,-19.15,H-SC,58.33,91965.0,-6015.0,28748.0,-0.36,-6.14,31.26,23.21,137.0,-0.21,0.67,33.38,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,8.44,X-SC,98.96,100912.0,-51.0,94585.0,0.01,-0.05,93.73,93.63,64.0,-0.00,0.74,50.31,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,1.31,M-LC,36.46,101842.0,23480.0,10551.0,-0.66,29.96,10.36,43.43,77.0,2.23,0.75,36.88,XY24,NTT,MISC
90,VALIANTORG,512.64,838.00,-355.15,H-SC,4.17,108420.0,-24866.0,109461.0,0.16,-18.66,100.96,63.47,148.0,-0.23,0.79,81.74,XR,NTT,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,142448.0,606.0,55968.0,0.05,0.43,39.29,39.88,259.0,0.01,1.04,44.36,XY24,NTT,POWER
4,ALKYLAMINE,2347.98,4546.37,8.44,X-SC,98.96,100912.0,-51.0,94585.0,0.01,-0.05,93.73,93.63,64.0,-0.00,0.74,50.31,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-2.21,H-SC,97.92,183350.0,18680.0,50403.0,4.41,11.34,27.49,41.95,121.0,0.37,1.34,19.66,X40N,ATH,AC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68122.0,-45427.0,85479.0,-0.11,-40.01,125.48,35.27,261.0,-0.53,0.50,66.73,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-212.13,H-SC,95.83,80896.0,-14420.0,14456.0,-1.23,-15.13,17.87,0.04,154.0,-1.00,0.59,59.56,OX40N,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.94
1,25,42.00
2,50,70.50


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.43
LC    33.98
MC    21.60
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.53
X40      12.24
XY25     12.03
X40N     11.88
XR       10.08
AR        8.22
OX40N     8.22
MH        1.70
X5K       1.48
X200      1.40
SR        1.23
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.61
M-SC    15.97
H-LC    14.12
X-LC    12.16
H-MC     9.54
X-MC     7.60
M-LC     6.65
X-SC     4.41
M-MC     4.10
L-SC     2.44
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.07,-1.54,35.48
IT,12.12,-9.69,63.58
BANKS,7.86,-6.21,49.54
FINANCE,7.86,-12.30,52.95
MISC,6.61,-15.15,68.58
PAINTS,5.74,-11.30,33.24
ELECTRICAL,5.52,2.35,43.26
HEALTHCARE,4.56,-3.39,33.59
AUTO,4.36,-13.63,63.59


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2693794.0,27
XR,1079143.0,13
AR,1021257.0,10
X40,646222.0,9
X40N,538946.0,11
XY25,475568.0,8
OX40N,308484.0,11
SR,175838.0,2
MH,84476.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1969297.0,20
M-SC,1760188.0,21
H-LC,610130.0,12
H-MC,529972.0,8
X-LC,513514.0,7
X-MC,413040.0,6
M-MC,409829.0,3
X-SC,315525.0,5
M-LC,265882.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      867667.0      6
M-SC       XY24      815513.0      7
H-SC       AR        522523.0      3
M-SC       XR        332786.0      4
M-MC       XY24      299168.0      2
H-MC       XY24      295893.0      3
X-LC       X40       239976.0      2
H-SC       XR        217794.0      3
           X40N      216296.0      4
H-LC       X40N      197881.0      4
M-SC       AR        181660.0      2
           OX40N     162940.0      5
M-LC       XY24      158263.0      3
H-LC       X40       154728.0      3
L-SC       XR        147638.0      2
X-MC       X40       147362.0      2
X-SC       XR        123107.0      1
X-LC       AR        116307.0      1
           XY25      115483.0      3
M-SC       XY25      114923.0      1
M-MC       XR        110661.0      1
X-MC       XY25      102663.0      1
           XY24       94923.0      1
X-SC       SR         94585.0      1
M-LC       XR         93581.0      1
H-SC       OX40N      89699.0      3
H-MC       XY25       81970.0      1
M-SC       SR         81253.0      1
H-LC       AR         79677.0      1
M-SC       X40        71113.0      1
H-LC       X200       68450.0      1
           X5K        66564.0      2
H-MC       AR         59848.0      1
L-SC       AR         59841.0      1
X-SC       XY24       57119.0      1
L-SC       XY24       55968.0      1
H-SC       MH         55318.0      1
L-MC       XR         53576.0      1
X-MC       X40N       52961.0      1
L-LC       XY25       46491.0      1
H-LC       XY24       42830.0      1
X-LC       X40N       41748.0      1
X-SC       OX40N      40714.0      2
H-MC       X40        33043.0      1
           X40N       30060.0      1
           MH         29158.0      1
X-MC       OX40N      15131.0      1
M-LC       XY25       14038.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 67.0 seconds
